Go through the iris_ml.ipnyb summary for most of the background.

Going off of that file, using "Keras" (which is based on either theano or, in my case, tensorflow) for deep-learning simplifies the programmer's job tons.

Setup
---

I'm having so much trouble with missing and incompatible Keras imports. Thankfully "scikeras" exists; it's some sort of library that makes working between Keras and scikit smoother.

In [ ]:
# !pip install scikeras

In [ ]:
import pandas

from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

from sklearn import datasets
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

There are format issues I was struggling with to get the scikit iris dataframe to work with keras, so I just downloaded the csv file [online](https://archive.ics.uci.edu/dataset/53/iris), and called the file from the directory. I use LabelEncoder() and to_categorical() to turn the Iris class data into a dummy_variable.

split the data like usual.

In [ ]:
dataframe = pandas.read_csv("iris.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]

encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
dummy_Y = to_categorical(encoded_Y)

X_train, X_validation, Y_train, Y_validation = train_test_split(X, dummy_Y, test_size=0.20, random_state=1)

Deep-Learning
---

This is the meat 'n potatoes here:

As I understand it, these DL models (I don't know that this is a common acronym), stand on neural networks as background data structure, inspired by the human brain and Cognitive Science. They are more sophisticated than traditional ML models, and I think you can say "cover their bases". I'll learn more about these when I get to take CSE 151B, but the important part is I know how to train them.

In [ ]:
def baseline_model():
  model = Sequential()
  model.add(Dense(8, input_dim=4, activation='relu'))
  model.add(Dense(3, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

Proceeding to train the deep-learning model:

Interestingly, an alternative to calling fit(), is to just throw the whole dataset into cv_score. But cv_score doesn't return a whole "classification report" like .fit() does and lacks metrics like precision, recall f1-score etc.

In [ ]:
estimator = KerasClassifier(model=baseline_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)

cv_score = cross_val_score(estimator, X, dummy_Y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (cv_score.mean()*100, cv_score.std()*100))

Baseline: 97.33% (3.27%)


In [ ]:
estimator.fit(X_train, Y_train)
predictions = estimator.predict(X_validation)
print(classification_report(Y_validation, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       1.00      0.92      0.96        13
           2       0.86      1.00      0.92         6

   micro avg       0.97      0.97      0.97        30
   macro avg       0.95      0.97      0.96        30
weighted avg       0.97      0.97      0.97        30
 samples avg       0.97      0.97      0.97        30



Interestingly not only did the DL training take much longer than traditional ML training, the results were not actually better. This is because 1. the Iris dataset is not actually very nuanced, so there are simply outliers that it would not necessarily be reasonable to expect the models to catch, & 2. in general the more sophisticated the heuristics (fancy word for guessing pattern) are, the more data would be required to generalize and find patterns.
In other words, point 2 would mean that the DL model is "overthinking it" and amazingly seeing connections that aren't there (which humans are very prone to do). This is called overfitting.

End of notebook